### Groupby
- split -> apply -> combine: 과정을 거쳐서 연산

In [46]:
import pandas as pd
import numpy as np
import dateutil
import sqlite3

In [2]:
# data from: 
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [3]:
df.groupby("Team")["Points"].sum()

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

### Hirarchical index
- unstack(): Group으로 묶여진 데이터를 matrix 형태로 전환해줌
- swaplevel(): index level을 변경할 수 있음

In [4]:
h_index = df.groupby(["Team", "Year"])["Points"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [5]:
h_index.index

MultiIndex([('Devils', 2014),
            ('Devils', 2015),
            ( 'Kings', 2014),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015),
            ( 'kings', 2015)],
           names=['Team', 'Year'])

In [6]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,NaN,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN
kings,NaN,812.0,NaN,NaN


In [7]:
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
      kings     812
Name: Points, dtype: int64

In [8]:
h_index.sum(level = 0)

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

In [9]:
h_index.sum(level=1)

Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64

### Groupyby - grouped
- groupby에 의해 split된 상태를 추출 가능함

In [11]:
grouped = df.groupby("Team")

for name, group in grouped:
    print(name)
    print(group)

Devils
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
Kings
    Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788
Riders
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
Royals
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804
kings
    Team  Rank  Year  Points
5  kings     4  2015     812


In [12]:
grouped.get_group("Devils") #특정 key 값을 가진 그룹의 정보만 추출 가능

,Team,Rank,Year,Points
2,Devils,2,2014,863
3,Devils,3,2015,673


### Groupby - grouped
- 추출된 group 정보에는 세 가지 유형의 apply가 가능함
- Aggregation: 요약된 통계 정보를 추출해 줌
- Transformation: 해당 정보를 변환해줌, Aggretagion과 달리 key 값 별로 요약된 정보 X, 개별 데이터 변환 지원
- Filteration: 특정 정보를 제거하여 보여주는 필터링 기능, 데이터 검색 시 사용

In [13]:
grouped.agg(sum)

,Rank,Year,Points
Team,,,
Devils,5,4029,1536
Kings,5,6047,2285
Riders,7,8062,3049
Royals,5,4029,1505
kings,4,2015,812


In [15]:
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
Devils,2.500000,2014.500000,768.000000
Kings,1.666667,2015.666667,761.666667
Riders,1.750000,2015.500000,762.250000
Royals,2.500000,2014.500000,752.500000
kings,4.000000,2015.000000,812.000000


In [16]:
grouped["Points"].agg([np.sum, np.mean, np.std])

,sum,mean,std
Team,,,
Devils,1536,768.000000,134.350288
Kings,2285,761.666667,24.006943
Riders,3049,762.250000,88.567771
Royals,1505,752.500000,72.831998
kings,812,812.000000,NaN


In [18]:
score = lambda x:(x-x.mean())/x.std() #standardization
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,1.154701,-1.091089,-0.860862
5,NaN,NaN,NaN
6,-0.577350,0.218218,-0.236043
7,-0.577350,0.872872,1.096905
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


In [19]:
df.groupby('Team').filter(lambda x:len(x)>=3)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [22]:
df_phone=pd.read_csv("https://www.shanelynn.ie/wp-content/uploads/2015/06/phone_data.csv")
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [23]:
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [24]:
df_phone.groupby("month")['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [25]:
df_phone[df_phone['item']=='call'].groupby('month')['duration'].sum()

month
2014-11    25547.0
2014-12    13561.0
2015-01    17070.0
2015-02    14416.0
2015-03    21727.0
Name: duration, dtype: float64

In [26]:
df_phone.groupby(['month', 'item']).agg({'duration':sum,      # find the sum of the durations for each group
                                     'network_type': "count", # find the number of network type entries
                                     'date': 'first'})    # get the first date per group

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [27]:
df_phone.groupby(['month', 'item']).agg({'duration': [min],      # find the min, max, and sum of the duration column
                                     'network_type': "count", # find the number of network type entries
                                     'date': [min, 'first', 'nunique']})    # get the min, first, and number of unique dates

duration network_type                date                      \
                  min        count                 min               first   
month   item                                                                 
2014-11 call    1.000          107 2014-10-15 06:58:00 2014-10-15 06:58:00   
        data   34.429           29 2014-10-15 06:58:00 2014-10-15 06:58:00   
        sms     1.000           94 2014-10-16 22:18:00 2014-10-16 22:18:00   
2014-12 call    2.000           79 2014-11-14 17:24:00 2014-11-14 17:24:00   
        data   34.429           30 2014-11-13 06:58:00 2014-11-13 06:58:00   
        sms     1.000           48 2014-11-14 17:28:00 2014-11-14 17:28:00   
2015-01 call    2.000           88 2014-12-15 20:03:00 2014-12-15 20:03:00   
        data   34.429           31 2014-12-13 06:58:00 2014-12-13 06:58:00   
        sms     1.000           86 2014-12-15 19:56:00 2014-12-15 19:56:00   
2015-02 call    1.000           67 2015-01-15 10:36:00 2015-01-15 10:36:00   
        data   34.429           31 2015-01-13 06:58:00 2015-01-13 06:58:00   
        sms     1.000           39 2015-01-15 12:23:00 2015-01-15 12:23:00   
2015-03 call    2.000           47 2015-02-12 20:15:00 2015-02-12 20:15:00   
        data   34.429           29 2015-02-13 06:58:00 2015-02-13 06:58:00   
        sms     1.000           25 2015-02-19 18:46:00 2015-02-19 18:46:00   

                      
             nunique  
month   item          
2014-11 call     104  
        data      29  
        sms       79  
2014-12 call      76  
        data      30  
        sms       41  
2015-01 call      84  
        data      31  
        sms       58  
2015-02 call      67  
        data      31  
        sms       27  
2015-03 call      47  
        data      29  
        sms       17

In [28]:
grouped = df_phone.groupby('month').agg( {"duration" : [min, max, np.mean]})
grouped

duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891

In [30]:
df_phone.pivot_table(['duration'],
                    index = [df_phone.month, df_phone.item],
                    columns = df_phone.network, aggfunc = "sum", fill_value = 0)

duration                                                   \
network        Meteor Tesco  Three Vodafone      data landline special   
month   item                                                             
2014-11 call     1521  4045  12458     4316     0.000     2906       0   
        data        0     0      0        0   998.441        0       0   
        sms        10     3     25       55     0.000        0       1   
2014-12 call     2010  1819   6316     1302     0.000     1424       0   
        data        0     0      0        0  1032.870        0       0   
        sms        12     1     13       18     0.000        0       0   
2015-01 call     2207  2904   6445     3626     0.000     1603       0   
        data        0     0      0        0  1067.299        0       0   
        sms        10     3     33       40     0.000        0       0   
2015-02 call     1188  4087   6279     1864     0.000      730       0   
        data        0     0      0        0  1067.299        0       0   
        sms         1     2     11       23     0.000        0       2   
2015-03 call      274   973   4966     3513     0.000    11770       0   
        data        0     0      0        0   998.441        0       0   
        sms         0     4      5       13     0.000        0       0   

                              
network      voicemail world  
month   item                  
2014-11 call       301     0  
        data         0     0  
        sms          0     0  
2014-12 call       690     0  
        data         0     0  
        sms          0     4  
2015-01 call       285     0  
        data         0     0  
        sms          0     0  
2015-02 call       268     0  
        data         0     0  
        sms          0     0  
2015-03 call       231     0  
        data         0     0  
        sms          0     3

### Cross tab

In [39]:
df_movie = pd.read_csv("https://raw.githubusercontent.com/TeamLab/machine_learning_from_scratch_with_python/master/code/ch5/data/movie_rating.csv")
pd.crosstab(index=df_movie.critic,columns=df_movie.title,values=df_movie.rating,
            aggfunc="first").fillna(0)

title,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,You Me and Dupree
critic,,,,,,
Claudia Puig,3.0,0.0,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,0.0,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,0.0,0.0,4.5,4.0,0.0,1.0


In [40]:
df_movie.pivot_table(["rating"], index=df_movie.critic, columns=df_movie.title,
                     aggfunc="sum", fill_value=0)

rating                                      \
title         Just My Luck Lady in the Water Snakes on a Plane   
critic                                                           
Claudia Puig           3.0               0.0               3.5   
Gene Seymour           1.5               3.0               3.5   
Jack Matthews          0.0               3.0               4.0   
Lisa Rose              3.0               2.5               3.5   
Mick LaSalle           2.0               3.0               4.0   
Toby                   0.0               0.0               4.5   

                                                                     
title         Superman Returns The Night Listener You Me and Dupree  
critic                                                               
Claudia Puig               4.0                4.5               2.5  
Gene Seymour               5.0                3.0               3.5  
Jack Matthews              5.0                3.0               3.5  
Lisa Rose                  3.5                3.0               2.5  
Mick LaSalle               3.0                3.0               2.0  
Toby                       4.0                0.0               1.0

### Merge
- SQL에서 많이 사용하는 Merge와 같은 기능
- 두 개의 데이터를 하나로 합침

In [41]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_score': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'test_score'])
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])

In [42]:
pd.merge(df_a, df_b, on='subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [43]:
pd.merge(df_a, df_b, right_index = True, left_index = True) #index based join

,subject_id_x,test_score,subject_id_y,first_name,last_name
0,1,51,4,Billy,Bonder
1,2,15,5,Brian,Black
2,3,15,6,Bran,Balwner
3,4,61,7,Bryce,Brice
4,5,16,8,Betty,Btisan


### Concat

In [44]:
df_new = pd.concat([df_a, df_b])
df_new.reset_index()

,index,subject_id,test_score,first_name,last_name
0,0,1,51.0,NaN,NaN
1,1,2,15.0,NaN,NaN
2,2,3,15.0,NaN,NaN
3,3,4,61.0,NaN,NaN
4,4,5,16.0,NaN,NaN
5,5,7,14.0,NaN,NaN
6,6,8,15.0,NaN,NaN
7,7,9,1.0,NaN,NaN
8,8,10,61.0,NaN,NaN
9,9,11,16.0,NaN,NaN


## Database connection
- Data loading 시 db connection 기능을 제공함

In [50]:
conn = sqlite3.connect("./data/flights.db") #Database 연결 코드
cur = conn.cursor()
cur.execute("select * from airlines limit 5;")
results = cur.fetchall()
results

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [52]:
#db 연결 conn을 사용하여 dataframe 생성
df_airplanes = pd.read_sql_query("select * from airlines;",conn) 
df_airports = pd.read_sql_query("select * from airports;",conn)
df_routes = pd.read_sql_query("select * from routes;", conn)

### XLS persistence
- Dataframe의 엑셀 추출 코드
- XLS 엔진으로 openpyxls 또는 XlsxWrite 사용

In [ ]:
writer = pd.ExcelWriter("./Data/df_routes/.xlsx", engine = 'xlsxwirter')
df_routes.to_excel(writer, sheet_name = "Sheet1")

### Pickle persistence
- 가장 일반적인 python 파일 persistence

In [ ]:
df_routes.to_pickle("./data/df_routes.pickle")